In [ ]:
import pandas as pd
import re

In [ ]:
disease_codes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/d_icd_diagnoses.csv")
patients_info = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/diagnoses_icd.csv")
pattern = r'\b(?:malignant\s+(?=.*\b(?:colon|rect|rectum|rectal|anal|anus|hepatic flexure|cecum|appendix vermiformis|flexure|large intestine)\b)|.*\b(?:colon|rect|rectum|rectal|anal|anus|hepatic flexure|cecum|appendix vermiformis|flexure|large intestine)\b\s+malignant\b)'
crc_codes = disease_codes[disease_codes["long_title"].str.contains(pattern, case=False, regex=True) & ~disease_codes["long_title"].str.contains("skin")]
# Filtering the DataFrame using isin() function
filtered_info = patients_info[patients_info['icd_code'].isin(crc_codes.icd_code.unique())]
filtered_info.subject_id.unique()

In [ ]:
genie_data = pd.read_table("/mnt/cbib/EOSC4Cancer/GENIE_CBioPortal/data_mutations_extended.txt")
genie_crc_subjects = pd.read_csv("/mnt/cbib/EOSC4Cancer/GENIE_CBioPortal/crc_public_genie_bpc_clinical_data.tsv", sep="\t")
genie_crc_subjects = genie_crc_subjects[genie_crc_subjects["Sex"]== "Male"]
genie_data = genie_data[genie_data.Tumor_Sample_Barcode.isin(genie_crc_subjects["Sample ID"].unique())]

In [ ]:
mimic_discharge_notes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv")
mimic_discharge_notes = mimic_discharge_notes[mimic_discharge_notes.subject_id.isin(filtered_info.subject_id.unique())]
mimic_radiology_notes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/radiology.csv")
mimic_radiology_notes = mimic_radiology_notes[mimic_radiology_notes.subject_id.isin(filtered_info.subject_id.unique())]

In [ ]:
genie_data[["Tumor_Sample_Barcode"]]

import pandas as pd
import random

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
grouped_df = genie_data.groupby('Tumor_Sample_Barcode')

# Selecting random elements from the group
num_random_selections = 500  # Change this number to the desired number of random selections

# Select random groups (not individual elements) based on the groups in 'Group_Column'
group_keys = list(grouped_df.groups.keys())
random_group_keys = random.sample(group_keys, num_random_selections)

# Filter the original DataFrame based on the randomly selected groups
genie_result_df = pd.concat([grouped_df.get_group(key) for key in random_group_keys])


In [ ]:
import pandas as pd
import random

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
grouped_df = mimic_discharge_notes.groupby('subject_id')

# Selecting random elements from the group
num_random_selections = 500  # Change this number to the desired number of random selections

# Select random groups (not individual elements) based on the groups in 'Group_Column'
group_keys = list(grouped_df.groups.keys())
random_group_keys = random.sample(group_keys, num_random_selections)

# Filter the original DataFrame based on the randomly selected groups
mimic_result_df = pd.concat([grouped_df.get_group(key) for key in random_group_keys])
pattern = r'Sex:\s*M'
mimic_result_df = mimic_result_df[mimic_result_df["text"].str.contains(pattern, case=False, regex=True)]
radiology_result_df = mimic_radiology_notes[mimic_radiology_notes["subject_id"].isin(mimic_result_df["subject_id"].unique())]


In [ ]:
import os

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
clinical_grouped_df = mimic_result_df.groupby('subject_id')
radiology_grouped_df = radiology_result_df.groupby('subject_id')

all_zipped_data = []

for key, group1 in clinical_grouped_df:
    if key in radiology_grouped_df.groups:
        group2 = radiology_grouped_df.get_group(key)
        zipped_data = zip(group1['subject_id'], group1['text'], group2['text'])
        all_zipped_data.extend(list(zipped_data))
        # Instead of iterating through zipped_data here, it's better to accumulate it in all_zipped_data
names= []
# Iterate through groups and save to separate CSV files
for data in all_zipped_data:
    name, clinical_text, radiology_text = data
    names.append(name)
    new_directory = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/"
    os.makedirs(new_directory, exist_ok=True)
    file_name = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/clinical_note.txt"  # Naming files based on group names
    with open(file_name, 'w') as file:
        # Write data to the file
        file.write(clinical_text)
        file.write(radiology_text)

In [ ]:
group1

In [ ]:
grouped_df = genie_result_df.groupby("Tumor_Sample_Barcode")
for name, (_, row) in zip(names, grouped_df):
    file_name = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/mutations_data.csv"
    row.to_csv(file_name)

In [ ]:
import re
# remove annoying characters
chars = {
    '\xc2\x82' : ',',        # High code comma
    '\xc2\x84' : ',,',       # High code double comma
    '\xc2\x85' : '...',      # Tripple dot
    '\xc2\x88' : '^',        # High carat
    '\xc2\x91' : '\x27',     # Forward single quote
    '\xc2\x92' : '\x27',     # Reverse single quote
    '\xc2\x93' : '\x22',     # Forward double quote
    '\xc2\x94' : '\x22',     # Reverse double quote
    '\xc2\x95' : ' ',
    '\xc2\x96' : '-',        # High hyphen
    '\xc2\x97' : '--',       # Double hyphen
    '\xc2\x99' : ' ',
    '\xc2\xa0' : ' ',
    '\xc2\xa6' : '|',        # Split vertical bar
    '\xc2\xab' : '<<',       # Double less than
    '\xc2\xbb' : '>>',       # Double greater than
    '\xc2\xbc' : '1/4',      # one quarter
    '\xc2\xbd' : '1/2',      # one half
    '\xc2\xbe' : '3/4',      # three quarters
    '\xca\xbf' : '\x27',     # c-single quote
    '\xcc\xa8' : '',         # modifier - under curve
    '\xcc\xb1' : '',          # modifier - under line
    '\x95' : ' ',
}
def replace_chars(match):
    char = match.group(0)
    return chars[char]

def remove_annoying_chars(text):
    pattern = '(' + '|'.join(chars.keys()) + ')'
    return re.sub(pattern, replace_chars, text)

# Example usage:
input_text = "Your input text here with annoying characters..."
result_text = remove_annoying_chars(input_text)
print(result_text)

In [ ]:
remove_annoying_chars(mimic_discharge_notes.iloc[10]["text"])

In [ ]:
mimic_discharge_notes.iloc[10]["text"].replace("\n\n", "").replace("\n", "").replace("___", "")

In [ ]:
from preprocessing_utils import split_text_to_sentences

In [ ]:
import utils
REGEX_FILE  = utils.load_regex_patterns("../data/regex_patterns.json")
wh = split_text_to_sentences(mimic_discharge_notes.iloc[0]["text"].replace("\n \n", "\n\n").replace("_", "\n\n").replace("===", '\n\n'), REGEX_FILE)

In [ ]:
with open('/mnt/cbib/EOSC4Cancer/synthetic_data/10669816/clinical_note.txt') as f:
    lines = f.read()

In [ ]:
lines

In [30]:
import preprocess_clinical_notes

In [62]:
df = preprocess_clinical_notes.tokenize_clinical_note(id_list=["13791943"], source_directory="/mnt/cbib/EOSC4Cancer/synthetic_data")
# df.to_csv("/mnt/cbib/EOSC4Cancer/synthetic_data/14014677/clinical_note.csv")

In [63]:
fields = ["Sex", "Allergies", "Cheif Complaint", "History of Present Illness", 
          "Major Surgical or Invasive Procedure", "Past Medical History", "Family History", "IMAGING", 
          "Brief Hospital Course", "Medications on Admission", "Discharge Medications"]
df = df[df["field"].isin(fields)].dropna()

In [64]:
df[["field", "sentence"]]

,field,sentence
5,Sex,M
7,Allergies,Patient recorded as having No Known Allergies...
10,Major Surgical or Invasive Procedure,None
11,History of Present Illness,
12,History of Present Illness,__
13,History of Present Illness,is a
14,History of Present Illness,__
15,History of Present Illness,year old right handed male presenting with R h...
16,History of Present Illness,He was last seen or heard from two days ago (
17,History of Present Illness,__


In [59]:
result_dict = df.groupby('field', sort=False)['sentence'].apply(list).to_dict()

In [61]:
result_dict

{'Sex': ['  F'],
 'Allergies': [' Influenza Virus Vaccines'],
 'Major Surgical or Invasive Procedure': [''],
 'History of Present Illness': ['',
  'yo F w/ endometrial CA w/ met to abd wall radiation colitis bladder CA s/p resection & BCG HTN hypothyroidism h/o L MCA thrombotic CVA w/o residual deficits enterovesical fistula s/p b/l nephrostomy tubes & ostomy who presents with decreased oral intake for over 1 week and right sided neck pain',
  'She states  that she had increasing pain that caused her to take Oxycodone 5mg',
  'BID rather than just once a day',
  'She has chronic dry mouth also takes a lot of medications with a common side effect of dry  mouth from all of them along with chronic poor dentition - which she has yet to see a dentist for',
  'Due to inability to eat for past 5 days son brought her to the ER to be evaluated',
  'She denies any fever chills abdominal pain difficulty swallowing',
  'She has +difficulty chewing due to pain has no dentures and + nausea after eat

In [60]:
import requests
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('/mnt/cbib/EOSC4Cancer/synthetic_data/', glob="**/*.txt")
documents = loader.load()

KeyboardInterrupt: 

In [ ]:
documents[0]

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(documents)

Created a chunk of size 1005, which is longer than the specified 1000
Created a chunk of size 1074, which is longer than the specified 1000
Created a chunk of size 1268, which is longer than the specified 1000
Created a chunk of size 1480, which is longer than the specified 1000
Created a chunk of size 1473, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 1476, which is longer than the specified 1000
Created a chunk of size 1435, which is longer than the specified 1000
Created a chunk of size 1260, which is longer than the specified 1000
Created a chunk of size 1043, which is longer than the specified 1000
Created a chunk of size 1356, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1084, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of s

In [ ]:
chunks

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = HuggingFaceEmbeddings(),
    by_text = False
)

/home/mabdallah/miniconda3/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-26T12:27:58+01:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-26T12:27:58+01:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-26T12:27:58+01:00"}
{"level":"warning","msg":"Multiple v

Started /home/mabdallah/.cache/weaviate-embedded: process ID 751358


looking up route for GET /v1/.well-known/openid-configuration
got a router for POST
got a router for DELETE
got a router for HEAD
got a router for PATCH
got a router for GET
got a router for PUT
found a route for GET /v1/.well-known/openid-configuration with 0 parameters
responding to GET /v1/.well-known/openid-configuration with produces: [application/json]
offers: [application/json]
[GET /v1/.well-known/openid-configuration] set response format "application/json" in context
[GET /v1/.well-known/openid-configuration] negotiated response format "application/json"
looking up route for GET /v1/meta
got a router for DELETE
got a router for HEAD
got a router for PATCH
got a router for GET
got a router for PUT
got a router for POST
found a route for GET /v1/meta with 0 parameters
responding to GET /v1/meta with produces: [application/json]
offers: [application/json]
[GET /v1/meta] set response format "application/json" in context
[GET /v1/meta] negotiated response format "application/json"


In [7]:
retriever = vectorstore.as_retriever()

In [25]:
from langchain.prompts import ChatPromptTemplate

template = """You are an oncologist assistant for generating synthetic cancer patient profiles. 
Use the following pieces of retrieved contexts to generate a single clinically plausible synthetic colorectal cancer patient profile. 
The synthetic patient profile should the following :
- Age 
- Sex 
- History of Present condition
- Past Medical History
- Symptoms 
- Allergies 
- Past and Present Medications
- Treatment 
- Treatment Outcomes
- Family History 
If some information are not available, you can generate the missing data.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an oncologist assistant for generating synthetic cancer patient profiles. \nUse the following pieces of retrieved contexts to generate a single clinically plausible synthetic colorectal cancer patient profile. \nThe synthetic patient profile should the following :\n- Age \n- Sex \n- History of Present condition\n- Past Medical History\n- Symptoms \n- Allergies \n- Past and Present Medications\n- Treatment \n- Treatment Outcomes\n- Family History \nIf some information are not available, you can generate the missing data.\nQuestion: {question} \nContext: {context} \nAnswer:\n'))]


In [11]:
from getpass import getpass
import os

HUGGINGFACEHUB_API_TOKEN = getpass()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.llms import HuggingFaceEndpoint
import os


repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "Generate a synthetic patient profile for a female colorectal cancer patient whose age is 45 years old"
response = rag_chain.invoke(query)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
looking up route for POST /v1/graphql
got a router for GET
got a router for PUT
got a router for POST
got a router for DELETE
got a router for HEAD
got a router for PATCH
found a route for POST /v1/graphql with 1 parameters
validating content type for "application/json" against [application/json, application/yaml]
responding to POST /v1/graphql with produces: [application/json]
offers: [application/json]
[POST /v1/graphql] set response format "application/json" in context
[POST /v1/graphql] negotiated response format "application/json"


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/mabdallah/.cache/huggingface/token
Login successful


In [29]:
print(response)

Age: 45 years old
Sex: Female
History of Present Condition: Not available in the context
Past Medical History: depression, open appendectomy in __, open gallbladder in __, hysterectomy in __, diabetes mellitus, hypertension, hyperlipidemia, arthritis, shoulder pain, tendonitis, colon cancer (unknown stage and treatment), SLE
Symptoms: Not available in the context
Allergies: Not available in the context
Past and Present Medications: Not available in the context
Treatment: Colonic resection, chemotherapy (unknown type)
Treatment Outcomes: Not available in the context
Family History: Family history is notable for colon cancer in her mother, no inflammatory bowel disease and no other cancers. Maternal aunt had breast cancer at an unknown age. Father died of colon cancer in an unknown age and is believed to have had multiple polyps prior to his death. Sister has hypertension, GERD, and gets colonoscopies every 3-5 years due to a history of polyps. No children. No other family history of can

{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 89.66%, threshold set to 80.00%","path":"/home/mabdallah/.local/share/weaviate","time":"2024-02-26T12:40:30+01:00"}
{"action":"lsm_compaction","class":"LangChain_21bff2a0d3b848ae8fb1c3620b7ac98e","index":"langchain_21bff2a0d3b848ae8fb1c3620b7ac98e","level":"warning","msg":"compaction halted due to shard READONLY status","path":"/home/mabdallah/.local/share/weaviate/langchain_21bff2a0d3b848ae8fb1c3620b7ac98e/3Ulvr6pHW3dB/lsm","shard":"3Ulvr6pHW3dB","time":"2024-02-26T13:32:45+01:00"}
{"action":"lsm_compaction","class":"LangChain_b68820591eb3431b9fc2b57b52d1beee","index":"langchain_b68820591eb3431b9fc2b57b52d1beee","level":"warning","msg":"compaction halted due to shard READONLY status","path":"/home/mabdallah/.local/share/weaviate/langchain_b68820591eb3431b9fc2b57b52d1beee/xomHdqd0BPy0/lsm","shard":"xomHdqd0BPy0","time":"2024-02-26T13:32:45+01:00"}
{"action":"lsm_compaction","class":"LangChain_fdf078be6dcd451886c